# <font color='#fff200'> **Notebook Purpose and Introduction**

### **Purpose**
To clearly outline the three criteria that will be referenced throughout this paper.

### **Introduction**
Having predefined criteria allows for standardization in evaluating the quality and meaningfulness of music pieces in general. Below are the three criteria I have chosen.

**Criteria 1 ~ Tonality**: Tonality refers to the organization of pitches and harmonies around a central note, known as the tonic. In meaningful music, intentional tonality indicates a deliberate choice and adherence to a specific tonal center throughout the composition. This criterion assesses whether the music maintains a coherent tonal structure, such as major or minor keys, which can provide a sense of stability, direction, and emotional resonance to the listener.

**Criteria 2 ~ Meter**: Meter refers to the rhythmic framework or pulse that organizes musical patterns into regular beats or measures. In meaningful music, meter plays a crucial role in establishing rhythmic stability and providing a sense of groove or flow. This criterion evaluates whether the music maintains a consistent meter, such as 4/4 time or waltz time, and effectively utilizes rhythmic patterns to convey musical structure and coherence.

**Criteria 3 ~ Melodic Structure**: Melodic structure refers to how individual pitches are organized over time to form a coherent and memorable melody. It includes the shape of the melody (pitch contour), the distances between pitches (intervallic relationships), the rhythmic patterns, the division into musical phrases (phrasing), and the development of recurring motifs (motivic development).

# **Necessary Libraries and Installations**

In [ ]:
!pip install mido

In [ ]:
!pip install midi2audio

In [ ]:
!pip install pydub

In [ ]:
!sudo apt-get install fluidsynth

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
import mido
import soundfile as sf

from librosa import midi_to_hz
from midi2audio import FluidSynth
from mido import MidiFile, MidiTrack, tempo2bpm
from pydub import AudioSegment
from fractions import Fraction
from scipy.signal import find_peaks


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# **Criteria 1 - Tonality**

**Running Pitch Detection on Each Piece**

In [ ]:
monophonicDirectory = "/content/drive/Shareddrives/neotyagi-dataset/nottingham-dataset-master/MIDI/NottinghamTest/[3] Pitch Test"
noteInTuneTolerance = 7
pieceInTuneThreshold = 85

noteFrequencies = [
    65.41, 69.30, 73.42, 77.78, 82.41, 87.31, 92.50, 98.00, 103.83, 110.00, 116.54,
    123.47, 130.81, 138.59, 146.83, 155.56, 164.81, 174.61, 185.00, 196.00, 207.65,
    220.00, 233.08, 246.94, 261.63, 277.18, 293.66, 311.13, 329.63, 349.23, 369.99,
    392.00, 415.30, 440.00, 466.16, 493.88, 523.25, 554.37, 587.33, 622.25, 659.26,
    698.46, 739.99, 783.99, 830.61, 880.00, 932.33, 987.77, 1046.50, 1108.73, 1174.66,
    1244.51, 1318.51, 1396.91, 1479.98, 1567.98, 1661.22, 1760.00, 1864.66, 1975.53
]

def findClosestNoteFrequency(frequency):
    return min(noteFrequencies, key = lambda x: abs(x - frequency))

for filename in os.listdir(monophonicDirectory):
    totalNotes = 0
    if filename.endswith("TRUNCATED.mid"):
        filepath = os.path.join(monophonicDirectory, filename)
        filepathWAV = filepath[:-4] + '.wav'
        fs = FluidSynth()
        fs.midi_to_audio(filepath, filepathWAV)
        audioData, sampleRate = librosa.load(filepathWAV, sr=None)
        inTuneCount = 0

        frequencyEstimatesPYIN = librosa.pyin(audioData, fmin = librosa.note_to_hz('C2'), fmax = librosa.note_to_hz('C7'), fill_na = -1)
        frequencies = frequencyEstimatesPYIN[0]
        magnitudes = frequencyEstimatesPYIN[1]


        for i in range(0, len(frequencies), 5):
          frequency = frequencies[i]
          if frequency > 0:
            # print(f"This is the Frequency by PYIN: {frequency:.2f} Hz")
            closestNoteFrequency = findClosestNoteFrequency(frequency)
            # print(f"This is the closestNoteFrequency: {closestNoteFrequency:.2f} Hz")

            if abs(frequency - closestNoteFrequency) <= noteInTuneTolerance:
                inTuneCount += 1
            totalNotes+=1

        percentageInTune = (inTuneCount / totalNotes) * 100
        isPieceInTune = percentageInTune >= pieceInTuneThreshold

        print("")
        print(f"File: {filename}")
        print(f"Percentage of in-tune notes: {percentageInTune}%")
        print(f"The piece is{' ' if isPieceInTune else ' not '}in tune.\n")

# **Criteria 2 - Meter**

**Running Beat Detection on Each Piece**

In [ ]:
testDirectoryMeter = "/content/drive/Shareddrives/neotyagi-dataset/maestro-v3.0.0/MaestroTest/[4] Meter Test"

# This function checks if a piece has randomly distributed notes.
def hasRandomNotes(audioFile, maxStabilityThreshold = 0.07, maxNoteRepetitions = 5, maxNoteDistributionStability = 14):
    audioData, sampleRate = librosa.load(audioFile)

    cqt = librosa.cqt(y = audioData, sr = sampleRate) # Calculate the constant-Q chromagram [a representation of an audio signal that captures the energy distribution of different pitches over time].
    chromagram = librosa.amplitude_to_db(np.abs(cqt), ref=np.max) # Convert to decibels for easier analysis.

    # Calculating the note distribution stability
    noteDistributionStability = np.std(chromagram, axis=0).mean()

    onsetStrength = librosa.onset.onset_strength(y = audioData, sr = sampleRate) # Onset Strength: The magnitude of sudden changes in the audio signal [beginning of a note].
    tempo, _ = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate)
    _, beatTimes = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate, units = 'time') # Estimating the tempo and the beat times.

    meanTempoInterval = np.mean(np.diff(beatTimes))
    meanTempoIntervalStandardDeviation = np.std(np.diff(beatTimes))
    rhythmicStability = meanTempoIntervalStandardDeviation / meanTempoInterval

    noteOnsets = librosa.onset.onset_detect(y = audioData, sr = sampleRate) # Detects the beginning of notes and calculating the maximum note repetitions in the audio.
    noteIntervals = np.diff(noteOnsets)
    maxNoteReps = np.max(np.bincount(noteIntervals))

    if rhythmicStability >= maxStabilityThreshold or noteDistributionStability >= maxNoteDistributionStability or maxNoteReps >= maxNoteRepititions:
        return True, rhythmicStability, noteDistributionStability, maxNoteReps
    else:
        return False, rhythmicStability, noteDistributionStability, maxNoteReps

for filename in os.listdir(testDirectoryMeter):
    if filename.endswith(".wav"):
        filepath = os.path.join(testDirectoryMeter, filename)
        # filepathWAV = filepath[:-4] + '.wav'
        # fs = FluidSynth()
        # fs.midi_to_audio(filepath, filepath)
        hasRandom, rhythmStability, noteDistributionStability, maxNoteRepititions = hasRandomNotes(filepath)
        if hasRandom:
            print(f"{filename}: The piece has randomly distributed notes.")
        else:
            print(f"{filename}: The piece has a good rhythm with structured note distribution.")
        print(f"Rhythmic Stability: {rhythmStability:.2f}")                     # A lower value of rhythmic stability indicates more stable rhythm patterns | higher = less stable rhythm patterns.
        print(f"Note Distribution Stability: {noteDistributionStability:.2f}")  # A lower value of note distribution stability indicates even distribution across the pitch range | higher = less even distribution
        print(f"Maximum Note Repititons: {maxNoteRepititions:.2f}")

# **Criteria 3 - Melodic Structure**

**Running Melodic Structure Detection on Each Piece**

In [ ]:
testDirectoryMelody = "/content/drive/Shareddrives/neotyagi-dataset/maestro-v3.0.0/MaestroTest/[6] Melodic Structure Test"

# Function to analyze melodic structure
def analyzeMelodicStructure(audioFile):
    audioData, samplingRate = librosa.load(audioFile)

    pitches, magnitudes = librosa.core.piptrack(y = audioData, sr = samplingRate) # Compute the pitch (fundamental frequency - lowest frequency) over time

    pitchContour = pitches[np.argmax(magnitudes, axis=0)]
    pitchContour = pitchContour.ravel()  # Flatten to a 1D Array.

    melodicIntervals = np.diff(pitchContour)                                    # Melodic intervals are the differences between consecutive pitch values.
    melodicPeaks, _ = find_peaks(pitchContour)
    melodicComplexity = np.std(melodicIntervals)                                # Measures the standard deviation of the intervals between consecutive pitches in the melodic contour.


    melodicStabilityRange = np.max(pitchContour) - np.min(pitchContour)         # Measures the span between the highest and lowest pitches in the melodic contour.
    noteDurations = np.diff(melodicPeaks) / samplingRate
    dynamicVariation = np.std(audioData)

    return melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation

for filename in os.listdir(testDirectoryMelody):
    if filename.endswith(".wav"):
        filepath = os.path.join(testDirectoryMelody, filename)
        melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation = analyzeMelodicStructure(filepath)  # Analyze melodic structure

        print(f'File: {filename}\n')

        print(f'Melodic Peaks: {melodicPeaks}')                                 # Peaks in the pitch contour can indicate potential melodic peaks or points of repetition.


        print(f'\nMelodic Complexity: {melodicComplexity}')                     # A higher melodic complexity indicates greater variation in the intervals between consecutive pitches.
                                                                                # This could suggest a more intricate and diverse melodic structure with complex patterns and variations.

                                                                                # A lower melodic complexity suggests more consistent intervals between pitches.
                                                                                # This could indicate a simpler or more repetitive melodic structure with less variation in pitch intervals.

        print(f'\nMelodic Stability: {melodicStabilityRange}')                  # A larger melodic stability range suggests a wider pitch range in the melody, indicating a more diverse or expansive melodic structure.
                                                                                # A smaller melodic stability range suggests a more focused or narrow pitch range, possibly indicating a simpler or repetitive melodic structure.

        print(f'\nAverage Note Duration: {np.mean(noteDurations):.4f} seconds') # A higher average note duration indicates that the notes in the melody are sustained for a longer time. This might suggest a slower-paced melodic style.
                                                                                # A lower average note duration suggests shorter note durations on average, which might indicate a faster-paced or more staccato melodic style.

        print(f'\nStandard Deviation of Note Duration: {np.std(noteDurations):.4f} seconds') # A higher standard deviation of note duration indicates greater variability in the duration of notes. This suggests a more dynamic melody.
                                                                                # A lower standard deviation suggests more consistent note durations, indicating a more regular and predictable rhythm.

        print(f'\nDynamic Variation: {dynamicVariation}')                       # A higher dynamic variation suggests a wider range of loudness levels in the melody. This might indicate a more expressive and dynamically varied performance.
                                                                                # A lower dynamic variation suggests a more consistent loudness level, which might indicate a more controlled or uniform performance.
        print()
        print('=' * 50)
        print()
